To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [Gemma 3 blog](https://unsloth.ai/blog/gemma3) for what's new in Unsloth and our [Reasoning blog](https://unsloth.ai/blog/r1-reasoning) on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
"""%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm
# Install latest Hugging Face for Gemma-3!
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3"""

'%%capture\nimport os\nif "COLAB_" not in "".join(os.environ.keys()):\n    !pip install unsloth vllm\nelse:\n    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]\n    !pip install --no-deps unsloth vllm\n# Install latest Hugging Face for Gemma-3!\n!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3'

In [2]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121
!pip install unsloth

In [3]:
!pip install datasets -q

In [4]:
!pip install --no-deps git+https://github.com/huggingface/transformers@v4.49.0-Gemma-3

  Cloning https://github.com/huggingface/transformers (to revision v4.49.0-Gemma-3) to /tmp/pip-req-build-jg88b7j0
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-jg88b7j0
  Running command git checkout -q 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Resolved https://github.com/huggingface/transformers to commit 367bab469b0ef32017e2a0a0a5dbac5d36002f03
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.50.0.dev0-py3-none-any.whl size=10936468 sha256=05dc17178bc13e68c8694d12f9205f03286f0ac8d0f30b4ab0dba35192992aff
  Stored in directory: /tmp/pip-ephem-wheel-cache-kj0q04jv/wheels/c1/95/d1/7d800368ab25942bf88f88fe1716597fcc8d757043c9b48a2f
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.51.2
    Uninstalling transf

In [5]:
"""#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt"""

'#@title Colab Extra Install { display-mode: "form" }\n%%capture\nimport os\nif "COLAB_" not in "".join(os.environ.keys()):\n    !pip install unsloth vllm\nelse:\n    !pip install --no-deps unsloth vllm\n    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]\n    # Skip restarting message in Colab\n    import sys, re, requests; modules = list(sys.modules.keys())\n    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None\n    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo\n    !pip install sentencepiece protobuf datasets huggingface_hub hf_transfer\n\n    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy\n    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content\n    with open("vllm_requirements.txt", "wb") as file:\n        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n"

### Unsloth

`FastModel` supports loading nearly any model now! This includes Vision and Text models!

In [6]:
from unsloth import FastModel
import torch

fourbit_models = [
    # 4bit dynamic quants for superior accuracy and low memory use
    "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-12b-it-unsloth-bnb-4bit",
    "unsloth/gemma-3-27b-it-unsloth-bnb-4bit",

    # Other popular models!
    "unsloth/Llama-3.1-8B",
    "unsloth/Llama-3.2-3B",
    "unsloth/Llama-3.3-70B",
    "unsloth/mistral-7b-instruct-v0.3",
    "unsloth/Phi-4",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastModel.from_pretrained(
    model_name = "unsloth/gemma-3-1b-it",
    max_seq_length = 2048, # Choose any for long context!
    load_in_4bit = True,  # 4 bit quantization to reduce memory
    load_in_8bit = False, # [NEW!] A bit more accurate, uses 2x memory
    full_finetuning = False, # [NEW!] We have full finetuning now!
    # token = "hf_...", # use one if using gated models
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.3.19: Fast Gemma3 patching. Transformers: 4.50.0.dev0.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu121. CUDA: 7.5. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/1.00G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/233 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

We now add LoRA adapters so we only need to update a small amount of parameters!

In [7]:
model = FastModel.get_peft_model(
    model,
    finetune_vision_layers     = False, # Turn off for just text!
    finetune_language_layers   = True,  # Should leave on!
    finetune_attention_modules = True,  # Attention good for GRPO
    finetune_mlp_modules       = True,  # SHould leave on always!

    r = 8,           # Larger = higher accuracy, but might overfit
    lora_alpha = 8,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407,
)

Unsloth: Making `model.base_model.model.model` require gradients


<a name="Data"></a>
### Data Prep
We now use the `Gemma-3` format for conversation style finetunes. We use [Maxime Labonne's FineTome-100k](https://huggingface.co/datasets/mlabonne/FineTome-100k) dataset in ShareGPT style. Gemma-3 renders multi turn conversations like below:

```
<bos><start_of_turn>user
Hello!<end_of_turn>
<start_of_turn>model
Hey there!<end_of_turn>
```

We use our `get_chat_template` function to get the correct chat template. We support `zephyr, chatml, mistral, llama, alpaca, vicuna, vicuna_old, phi3, llama3, phi4, qwen2.5, gemma3` and more.

In [8]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)

In [9]:
"""from datasets import load_dataset
dataset = load_dataset('csv', data_files='/gdrive/MyDrive/nlp mental health conversations/translated_train.csv',split="train")
"""

'from datasets import load_dataset\ndataset = load_dataset(\'csv\', data_files=\'/gdrive/MyDrive/nlp mental health conversations/translated_train.csv\',split="train")\n'

In [10]:
"""from datasets import load_dataset

ds = load_dataset("vamsss/translated-16k-samples",split="train")
ds"""

'from datasets import load_dataset\n\nds = load_dataset("vamsss/translated-16k-samples",split="train")\nds'

In [11]:
"""#SHEN
from datasets import Dataset, load_dataset

# Step 1: Load the dataset from ShenLab/MentalChat16K
ds = load_dataset("vamsss/translated-16k-samples", split="train")

# Step 2: Transform the data into the desired chat format
transformed_data = []
for row in ds:
    # Combine 'instruction' and 'input' for the human entry
    human_content = row["instruction"]
    if row["input"]:  # Append input if it exists and is non-empty
        human_content += "\n" + row["input"]
    
    human_entry = {
        "from": "human",
        "value": human_content
    }
    gpt_entry = {
        "from": "gpt",
        "value": row["output"]
    }
    # Combine into a single conversation entry
    transformed_data.append([human_entry, gpt_entry])

# Step 3: Convert the transformed data into a flat structure for Dataset
flat_data = {"conversations": transformed_data}

# Step 4: Create a Dataset object from the transformed data
dataset = Dataset.from_dict(flat_data)

# Optional: Verify the first entry
print("First transformed entry:")
print(dataset[0])"""

'#SHEN\nfrom datasets import Dataset, load_dataset\n\n# Step 1: Load the dataset from ShenLab/MentalChat16K\nds = load_dataset("vamsss/translated-16k-samples", split="train")\n\n# Step 2: Transform the data into the desired chat format\ntransformed_data = []\nfor row in ds:\n    # Combine \'instruction\' and \'input\' for the human entry\n    human_content = row["instruction"]\n    if row["input"]:  # Append input if it exists and is non-empty\n        human_content += "\n" + row["input"]\n    \n    human_entry = {\n        "from": "human",\n        "value": human_content\n    }\n    gpt_entry = {\n        "from": "gpt",\n        "value": row["output"]\n    }\n    # Combine into a single conversation entry\n    transformed_data.append([human_entry, gpt_entry])\n\n# Step 3: Convert the transformed data into a flat structure for Dataset\nflat_data = {"conversations": transformed_data}\n\n# Step 4: Create a Dataset object from the transformed data\ndataset = Dataset.from_dict(flat_data)\n

In [12]:
from datasets import Dataset, load_dataset

# Step 1: Load the dataset
ds = load_dataset("vamsss/translated-16k-samples", split="train")  # Adjust path if different

# Step 2: Transform the data into the desired chat format
transformed_data = []
for i, row in enumerate(ds):
    # Ensure 'input' and 'output' are strings
    input_content = row["input"]
    output_content = row["output"]
    
    # Convert to string and handle edge cases
    if not isinstance(input_content, str):
        if input_content is None:
            input_content = ""
        else:
            input_content = str(input_content)
        print(f"Warning: Row {i} input converted to string: {input_content}")
    
    if not isinstance(output_content, str):
        if output_content is None:
            output_content = ""
        else:
            output_content = str(output_content)
        print(f"Warning: Row {i} output converted to string: {output_content}")
    
    human_entry = {
        "from": "human",
        "value": input_content
    }
    gpt_entry = {
        "from": "gpt",
        "value": output_content
    }
    # Combine into a single conversation entry
    transformed_data.append([human_entry, gpt_entry])

# Step 3: Convert the transformed data into a flat structure for Dataset
flat_data = {"conversations": transformed_data}

# Step 4: Create a Dataset object from the transformed data
dataset = Dataset.from_dict(flat_data)

# Optional: Verify the first entry
print("First transformed entry:")
print(dataset[0])

README.md:   0%|          | 0.00/316 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/42.4M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/16084 [00:00<?, ? examples/s]

First transformed entry:
{'conversations': [{'from': 'human', 'value': 'நான் இப்போது என் மன ஆரோக்கியத்துடன் போராடுகிறேன், அதைச் சமாளிக்க ஒரு வழியை என்னால் கண்டுபிடிக்க முடியவில்லை. நான் காட்சிப்படுத்தல், நேர்மறையான சிந்தனை மற்றும் மருந்துகளை கூட முயற்சித்தேன், ஆனால் எதுவும் செயல்படவில்லை. நான் தொலைந்துவிட்டேன், உதவியற்றவனாக உணர்கிறேன், அடுத்து என்ன செய்வது என்று எனக்குத் தெரியவில்லை. என் மனம் எண்ணங்கள் மற்றும் உணர்ச்சிகளின் சூறாவளி, அதையெல்லாம் என்னால் புரிந்து கொள்ள முடியாது. நான் குழப்பமான கடலில் மூழ்கி வருவதைப் போல உணர்கிறேன், என் வழியை என்னால் கண்டுபிடிக்க முடியவில்லை.'}, {'from': 'gpt', 'value': 'உங்கள் எண்ணங்களிலும் உணர்ச்சிகளிலும் குழப்பம் மற்றும் குழப்பம் ஆகியவற்றை நீங்கள் இப்போது சில காலமாக கையாள்கிறீர்கள் என்பதை நான் புரிந்துகொள்கிறேன். இது ஒரு சவாலான பயணமாகும், மேலும் உங்கள் அறிகுறிகளை நிர்வகிக்க காட்சிப்படுத்தல், நேர்மறையான சிந்தனை மற்றும் மருந்துகள் போன்ற பல்வேறு அணுகுமுறைகளை நீங்கள் முயற்சித்தீர்கள் என்பது பாராட்டத்தக்கது. இருப்பினும், இந்த முறைகள் உங்களுக்கு பயனுள்ளதாக இ

In [13]:
"""OG import pandas as pd
from datasets import Dataset, load_dataset

# Step 1: Load the CSV file into a pandas DataFrame
csv_path = '/kaggle/input/translated-train/translated_train.csv'
df = pd.read_csv(csv_path)

# Step 2: Transform the data into the desired chat format
transformed_data = []
for _, row in df.iterrows():
    # Assuming the first column is unnamed (index), and we use "Context" and "Response"
    human_entry = {
        "from": "human",
        "value": row["Context"]
    }
    gpt_entry = {
        "from": "gpt",
        "value": row["Response"]
    }
    # Combine into a single conversation entry
    transformed_data.append([human_entry, gpt_entry])

# Step 3: Convert the transformed data into a flat structure for Dataset
# We'll store each conversation as a single row with a "conversations" field
flat_data = {"conversations": transformed_data}

# Step 4: Create a Dataset object from the transformed data
dataset = Dataset.from_dict(flat_data)

# Optional: Verify the first entry
print("First transformed entry:")
print(dataset[0])

# Note: If you want to use load_dataset directly from CSV, you’d need to save the transformed data as JSON first.
# Alternatively, you can proceed with this Dataset object for fine-tuning."""

'OG import pandas as pd\nfrom datasets import Dataset, load_dataset\n\n# Step 1: Load the CSV file into a pandas DataFrame\ncsv_path = \'/kaggle/input/translated-train/translated_train.csv\'\ndf = pd.read_csv(csv_path)\n\n# Step 2: Transform the data into the desired chat format\ntransformed_data = []\nfor _, row in df.iterrows():\n    # Assuming the first column is unnamed (index), and we use "Context" and "Response"\n    human_entry = {\n        "from": "human",\n        "value": row["Context"]\n    }\n    gpt_entry = {\n        "from": "gpt",\n        "value": row["Response"]\n    }\n    # Combine into a single conversation entry\n    transformed_data.append([human_entry, gpt_entry])\n\n# Step 3: Convert the transformed data into a flat structure for Dataset\n# We\'ll store each conversation as a single row with a "conversations" field\nflat_data = {"conversations": transformed_data}\n\n# Step 4: Create a Dataset object from the transformed data\ndataset = Dataset.from_dict(flat_dat

In [14]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'நான் இப்போது என் மன ஆரோக்கியத்துடன் போராடுகிறேன், அதைச் சமாளிக்க ஒரு வழியை என்னால் கண்டுபிடிக்க முடியவில்லை. நான் காட்சிப்படுத்தல், நேர்மறையான சிந்தனை மற்றும் மருந்துகளை கூட முயற்சித்தேன், ஆனால் எதுவும் செயல்படவில்லை. நான் தொலைந்துவிட்டேன், உதவியற்றவனாக உணர்கிறேன், அடுத்து என்ன செய்வது என்று எனக்குத் தெரியவில்லை. என் மனம் எண்ணங்கள் மற்றும் உணர்ச்சிகளின் சூறாவளி, அதையெல்லாம் என்னால் புரிந்து கொள்ள முடியாது. நான் குழப்பமான கடலில் மூழ்கி வருவதைப் போல உணர்கிறேன், என் வழியை என்னால் கண்டுபிடிக்க முடியவில்லை.'},
  {'from': 'gpt',
   'value': 'உங்கள் எண்ணங்களிலும் உணர்ச்சிகளிலும் குழப்பம் மற்றும் குழப்பம் ஆகியவற்றை நீங்கள் இப்போது சில காலமாக கையாள்கிறீர்கள் என்பதை நான் புரிந்துகொள்கிறேன். இது ஒரு சவாலான பயணமாகும், மேலும் உங்கள் அறிகுறிகளை நிர்வகிக்க காட்சிப்படுத்தல், நேர்மறையான சிந்தனை மற்றும் மருந்துகள் போன்ற பல்வேறு அணுகுமுறைகளை நீங்கள் முயற்சித்தீர்கள் என்பது பாராட்டத்தக்கது. இருப்பினும், இந்த முறைகள் உங்களுக்கு பயனுள்ளதாக இல்லை என்பது தெளிவ

In [15]:
"""from datasets import load_dataset
dataset = load_dataset("mlabonne/FineTome-100k", split = "train")"""

'from datasets import load_dataset\ndataset = load_dataset("mlabonne/FineTome-100k", split = "train")'

In [16]:
dataset[0]

{'conversations': [{'from': 'human',
   'value': 'நான் இப்போது என் மன ஆரோக்கியத்துடன் போராடுகிறேன், அதைச் சமாளிக்க ஒரு வழியை என்னால் கண்டுபிடிக்க முடியவில்லை. நான் காட்சிப்படுத்தல், நேர்மறையான சிந்தனை மற்றும் மருந்துகளை கூட முயற்சித்தேன், ஆனால் எதுவும் செயல்படவில்லை. நான் தொலைந்துவிட்டேன், உதவியற்றவனாக உணர்கிறேன், அடுத்து என்ன செய்வது என்று எனக்குத் தெரியவில்லை. என் மனம் எண்ணங்கள் மற்றும் உணர்ச்சிகளின் சூறாவளி, அதையெல்லாம் என்னால் புரிந்து கொள்ள முடியாது. நான் குழப்பமான கடலில் மூழ்கி வருவதைப் போல உணர்கிறேன், என் வழியை என்னால் கண்டுபிடிக்க முடியவில்லை.'},
  {'from': 'gpt',
   'value': 'உங்கள் எண்ணங்களிலும் உணர்ச்சிகளிலும் குழப்பம் மற்றும் குழப்பம் ஆகியவற்றை நீங்கள் இப்போது சில காலமாக கையாள்கிறீர்கள் என்பதை நான் புரிந்துகொள்கிறேன். இது ஒரு சவாலான பயணமாகும், மேலும் உங்கள் அறிகுறிகளை நிர்வகிக்க காட்சிப்படுத்தல், நேர்மறையான சிந்தனை மற்றும் மருந்துகள் போன்ற பல்வேறு அணுகுமுறைகளை நீங்கள் முயற்சித்தீர்கள் என்பது பாராட்டத்தக்கது. இருப்பினும், இந்த முறைகள் உங்களுக்கு பயனுள்ளதாக இல்லை என்பது தெளிவ

We now use `standardize_data_formats` to try converting datasets to the correct format for finetuning purposes!

In [17]:
from unsloth.chat_templates import standardize_data_formats
dataset = standardize_data_formats(dataset)

Unsloth: Standardizing formats (num_proc=4):   0%|          | 0/16084 [00:00<?, ? examples/s]

Let's see how row 100 looks like!

In [18]:
"""#SHEN
from datasets import Dataset, load_dataset
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

# Step 1: Load the dataset from ShenLab/MentalChat16K
ds = load_dataset("ShenLab/MentalChat16K", split="train")

# Step 2: Transform the data into the chat format expected by Gemma-3
transformed_data = []
for row in ds:
    # Combine 'instruction' and 'input' for the user entry
    user_content = row["instruction"]
    if row["input"]:  # Append input if it exists and is non-empty
        user_content += "\n" + row["input"]
    
    user_entry = {
        "role": "user",  # Gemma-3 expects "role"
        "content": user_content  # Use "content" for consistency
    }
    assistant_entry = {
        "role": "assistant",
        "content": row["output"]
    }
    # Combine into a single conversation
    conversation = [user_entry, assistant_entry]
    transformed_data.append({"conversations": conversation})

# Step 3: Create a Dataset object
dataset = Dataset.from_list(transformed_data)


# Verify the first entry
print("First transformed entry:")
print(dataset[0])"""

'#SHEN\nfrom datasets import Dataset, load_dataset\nfrom unsloth import FastLanguageModel\nfrom unsloth.chat_templates import get_chat_template\n\n# Step 1: Load the dataset from ShenLab/MentalChat16K\nds = load_dataset("ShenLab/MentalChat16K", split="train")\n\n# Step 2: Transform the data into the chat format expected by Gemma-3\ntransformed_data = []\nfor row in ds:\n    # Combine \'instruction\' and \'input\' for the user entry\n    user_content = row["instruction"]\n    if row["input"]:  # Append input if it exists and is non-empty\n        user_content += "\n" + row["input"]\n    \n    user_entry = {\n        "role": "user",  # Gemma-3 expects "role"\n        "content": user_content  # Use "content" for consistency\n    }\n    assistant_entry = {\n        "role": "assistant",\n        "content": row["output"]\n    }\n    # Combine into a single conversation\n    conversation = [user_entry, assistant_entry]\n    transformed_data.append({"conversations": conversation})\n\n# Step 3:

In [19]:
from datasets import Dataset, load_dataset
#from unsloth import FastLanguageModel
#from unsloth.chat_templates import get_chat_template

# Step 1: Load the dataset
ds = load_dataset("vamsss/translated-16k-samples", split="train")  # Adjust path if different

# Step 2: Transform the data into the chat format expected by Gemma-3
transformed_data = []
for i, row in enumerate(ds):
    # Ensure 'input' and 'output' are strings
    input_content = row["input"]
    output_content = row["output"]
    
    # Convert to string and handle edge cases
    if not isinstance(input_content, str):
        if input_content is None:
            input_content = ""
        else:
            input_content = str(input_content)
        print(f"Warning: Row {i} input converted to string: {input_content}")
    
    if not isinstance(output_content, str):
        if output_content is None:
            output_content = ""
        else:
            output_content = str(output_content)
        print(f"Warning: Row {i} output converted to string: {output_content}")
    
    user_entry = {
        "role": "user",
        "content": input_content
    }
    assistant_entry = {
        "role": "assistant",
        "content": output_content
    }
    # Combine into a single conversation
    conversation = [user_entry, assistant_entry]
    transformed_data.append({"conversations": conversation})

# Step 3: Create a Dataset object
dataset = Dataset.from_list(transformed_data)

# Optional: Add chat template application (uncomment to use)
"""
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/gemma-3-1b-it",
    max_seq_length=2048,
    load_in_4bit=True,
)
tokenizer = get_chat_template(tokenizer, chat_template="gemma-3")

def apply_chat_template(examples):
    texts = [tokenizer.apply_chat_template(conv, tokenize=False, add_generation_prompt=True) for conv in examples["conversations"]]
    return {"text": texts}

dataset = dataset.map(apply_chat_template, batched=True)
print("First entry with chat template:")
print(dataset[0]["text"])
"""

# Verify the first entry
print("First transformed entry:")
print(dataset[0])

First transformed entry:
{'conversations': [{'content': 'நான் இப்போது என் மன ஆரோக்கியத்துடன் போராடுகிறேன், அதைச் சமாளிக்க ஒரு வழியை என்னால் கண்டுபிடிக்க முடியவில்லை. நான் காட்சிப்படுத்தல், நேர்மறையான சிந்தனை மற்றும் மருந்துகளை கூட முயற்சித்தேன், ஆனால் எதுவும் செயல்படவில்லை. நான் தொலைந்துவிட்டேன், உதவியற்றவனாக உணர்கிறேன், அடுத்து என்ன செய்வது என்று எனக்குத் தெரியவில்லை. என் மனம் எண்ணங்கள் மற்றும் உணர்ச்சிகளின் சூறாவளி, அதையெல்லாம் என்னால் புரிந்து கொள்ள முடியாது. நான் குழப்பமான கடலில் மூழ்கி வருவதைப் போல உணர்கிறேன், என் வழியை என்னால் கண்டுபிடிக்க முடியவில்லை.', 'role': 'user'}, {'content': 'உங்கள் எண்ணங்களிலும் உணர்ச்சிகளிலும் குழப்பம் மற்றும் குழப்பம் ஆகியவற்றை நீங்கள் இப்போது சில காலமாக கையாள்கிறீர்கள் என்பதை நான் புரிந்துகொள்கிறேன். இது ஒரு சவாலான பயணமாகும், மேலும் உங்கள் அறிகுறிகளை நிர்வகிக்க காட்சிப்படுத்தல், நேர்மறையான சிந்தனை மற்றும் மருந்துகள் போன்ற பல்வேறு அணுகுமுறைகளை நீங்கள் முயற்சித்தீர்கள் என்பது பாராட்டத்தக்கது. இருப்பினும், இந்த முறைகள் உங்களுக்கு பயனுள்ளதாக இல்லை என்பது 

In [20]:
"""OG import pandas as pd
from datasets import Dataset
from unsloth import FastLanguageModel
from unsloth.chat_templates import get_chat_template

# Step 1: Load the CSV file
csv_path = '/kaggle/input/translated-train/translated_train.csv'
df = pd.read_csv(csv_path)

# Step 2: Transform the data into the chat format expected by Gemma-3
transformed_data = []
for _, row in df.iterrows():
    user_entry = {
        "role": "user",  # Gemma-3 expects "role", not "from"
        "content": row["Context"]  # Use "content" for consistency
    }
    assistant_entry = {
        "role": "assistant",
        "content": row["Response"]
    }
    # Combine into a single conversation
    conversation = [user_entry, assistant_entry]
    transformed_data.append({"conversations": conversation})

# Step 3: Create a Dataset object
dataset = Dataset.from_list(transformed_data)


"""
"""# Step 5: Define the mapping function for batched processing
def apply_chat_template(examples):
    # Apply the chat template to each conversation in the batch
    texts = [tokenizer.apply_chat_template(conv, tokenize=False, add_generation_prompt=True) for conv in examples["conversations"]]
    return {"text": texts}

# Step 6: Apply the chat template to the dataset
dataset = dataset.map(apply_chat_template, batched=True)
"""
"""
# Verify the first entry
print("First transformed entry:")
print(dataset[0])"""

'\n# Verify the first entry\nprint("First transformed entry:")\nprint(dataset[0])'

In [21]:
dataset[2]

{'conversations': [{'content': 'நான் தொடர்ந்து ஆர்வத்துடன் உணர்கிறேன், ஒரு நேரத்தில் சில நிமிடங்களுக்கு மேல் எதையும் கவனம் செலுத்த முடியவில்லை. என்னால் எதையும் செய்து முடிக்கத் தெரியவில்லை, என்னை நம்பியிருக்கும் அனைவரையும் நான் தோல்வியுற்றதைப் போல உணர ஆரம்பிக்கிறேன்.',
   'role': 'user'},
  {'content': 'நீங்கள் மிகுந்த கவலையைக் கையாளுகிறீர்கள் என்பதையும், கவனம் செலுத்துவதற்கும் உற்பத்தி செய்வதற்கும் இது உங்கள் திறனை பாதிக்கிறது என்பதையும் என்னால் காண முடிகிறது. கவலை என்பது மன அழுத்தத்திற்கு ஒரு சாதாரண பதில் என்பதை நினைவில் கொள்வது முக்கியம், ஆனால் அது நாள்பட்டதாகி உங்கள் அன்றாட வாழ்க்கையில் தலையிடும்போது, \u200b\u200bஅது பலவீனமடையக்கூடும்.\n\nபதட்டத்தை நிர்வகிப்பதற்கான ஒரு அணுகுமுறை அறிவாற்றல்-நடத்தை நுட்பங்கள் மூலம். இது உங்கள் கவலைக்கு பங்களிக்கும் எதிர்மறை சிந்தனை முறைகள் அல்லது அறிவாற்றல் சிதைவுகளை அடையாளம் கண்டு சவால் செய்வது அடங்கும். எடுத்துக்காட்டாக, "என்னால் இதைச் செய்ய முடியாது" அல்லது "இந்த சூழ்நிலையை என்னால் ஒருபோதும் கையாள முடியாது" போன்ற விஷயங்களை நீங்கள் நினைத்துக்கொண்டி

In [22]:
dataset[100]

{'conversations': [{'content': 'இந்த நகரத்திற்குச் சென்றதிலிருந்து எனது நண்பர்களிடமிருந்து துண்டிக்கப்பட்டுள்ளதாக உணர்கிறேன். நான் கிட்டத்தட்ட ஒவ்வொரு நாளும் அவற்றைப் பார்த்தேன், இப்போது நான் அவர்களைப் பார்க்கவில்லை. இந்த புதிய சூழலை சரிசெய்வது கடினம், மேலும் எனது கல்லூரி நண்பர்கள் வழங்கிய சமூகத்தின் உணர்வையும் ஆதரவையும் நான் இழக்கிறேன்.',
   'role': 'user'},
  {'content': 'ஒரு வலுவான ஆதரவு அமைப்பை விட்டுவிட்டு புதிய இடத்தில் புதிதாகத் தொடங்குவது எவ்வளவு கடினம் என்பதை நான் புரிந்துகொள்கிறேன். உங்கள் கல்லூரி நண்பர்கள் உங்கள் வாழ்நாள் முழுவதும் உங்களுக்காக ஆறுதலையும் இணைப்பையும் பெற்றிருக்கிறார்கள், குறிப்பாக குடும்பம் குறைவாக அணுகக்கூடிய ஒரு சிறிய நகரத்தில் வளர்ந்து வந்ததிலிருந்து. உடல் தூரம் மற்றும் பிஸியான கால அட்டவணைகள் உங்களிடம் ஒரு முறை வைத்திருந்த அதே அளவிலான இணைப்பைப் பராமரிப்பது சவாலாக உள்ளது. இந்த உறவுகளின் முக்கியத்துவத்தையும், உங்கள் உணர்ச்சி நல்வாழ்வில் அவர்கள் வகிக்கும் பங்கையும் ஒப்புக்கொள்வது முக்கியம்.\n\nநாம் ஆராயக்கூடிய ஒரு மூலோபாயம், அடிக்கடி தொடர்புகொள்வதற்கான வழிகளை

We now have to apply the chat template for `Gemma-3` onto the conversations, and save it to `text`

In [23]:
def apply_chat_template(examples):
    texts = [tokenizer.apply_chat_template(conv, tokenize=False, add_generation_prompt=True) for conv in examples["conversations"]]
    #texts = tokenizer.apply_chat_template(examples["conversations"])
    return { "text" : texts }
pass
dataset = dataset.map(apply_chat_template, batched = True)

Map:   0%|          | 0/16084 [00:00<?, ? examples/s]

Let's see how the chat template did! Notice `Gemma-3` default adds a `<bos>`!

In [24]:
dataset[100]["text"]

'<bos><start_of_turn>user\nஇந்த நகரத்திற்குச் சென்றதிலிருந்து எனது நண்பர்களிடமிருந்து துண்டிக்கப்பட்டுள்ளதாக உணர்கிறேன். நான் கிட்டத்தட்ட ஒவ்வொரு நாளும் அவற்றைப் பார்த்தேன், இப்போது நான் அவர்களைப் பார்க்கவில்லை. இந்த புதிய சூழலை சரிசெய்வது கடினம், மேலும் எனது கல்லூரி நண்பர்கள் வழங்கிய சமூகத்தின் உணர்வையும் ஆதரவையும் நான் இழக்கிறேன்.<end_of_turn>\n<start_of_turn>model\nஒரு வலுவான ஆதரவு அமைப்பை விட்டுவிட்டு புதிய இடத்தில் புதிதாகத் தொடங்குவது எவ்வளவு கடினம் என்பதை நான் புரிந்துகொள்கிறேன். உங்கள் கல்லூரி நண்பர்கள் உங்கள் வாழ்நாள் முழுவதும் உங்களுக்காக ஆறுதலையும் இணைப்பையும் பெற்றிருக்கிறார்கள், குறிப்பாக குடும்பம் குறைவாக அணுகக்கூடிய ஒரு சிறிய நகரத்தில் வளர்ந்து வந்ததிலிருந்து. உடல் தூரம் மற்றும் பிஸியான கால அட்டவணைகள் உங்களிடம் ஒரு முறை வைத்திருந்த அதே அளவிலான இணைப்பைப் பராமரிப்பது சவாலாக உள்ளது. இந்த உறவுகளின் முக்கியத்துவத்தையும், உங்கள் உணர்ச்சி நல்வாழ்வில் அவர்கள் வகிக்கும் பங்கையும் ஒப்புக்கொள்வது முக்கியம்.\n\nநாம் ஆராயக்கூடிய ஒரு மூலோபாயம், அடிக்கடி தொடர்புகொள்வதற்கான வழிகளைக் கண்

In [25]:
dataset[100]["text"]

'<bos><start_of_turn>user\nஇந்த நகரத்திற்குச் சென்றதிலிருந்து எனது நண்பர்களிடமிருந்து துண்டிக்கப்பட்டுள்ளதாக உணர்கிறேன். நான் கிட்டத்தட்ட ஒவ்வொரு நாளும் அவற்றைப் பார்த்தேன், இப்போது நான் அவர்களைப் பார்க்கவில்லை. இந்த புதிய சூழலை சரிசெய்வது கடினம், மேலும் எனது கல்லூரி நண்பர்கள் வழங்கிய சமூகத்தின் உணர்வையும் ஆதரவையும் நான் இழக்கிறேன்.<end_of_turn>\n<start_of_turn>model\nஒரு வலுவான ஆதரவு அமைப்பை விட்டுவிட்டு புதிய இடத்தில் புதிதாகத் தொடங்குவது எவ்வளவு கடினம் என்பதை நான் புரிந்துகொள்கிறேன். உங்கள் கல்லூரி நண்பர்கள் உங்கள் வாழ்நாள் முழுவதும் உங்களுக்காக ஆறுதலையும் இணைப்பையும் பெற்றிருக்கிறார்கள், குறிப்பாக குடும்பம் குறைவாக அணுகக்கூடிய ஒரு சிறிய நகரத்தில் வளர்ந்து வந்ததிலிருந்து. உடல் தூரம் மற்றும் பிஸியான கால அட்டவணைகள் உங்களிடம் ஒரு முறை வைத்திருந்த அதே அளவிலான இணைப்பைப் பராமரிப்பது சவாலாக உள்ளது. இந்த உறவுகளின் முக்கியத்துவத்தையும், உங்கள் உணர்ச்சி நல்வாழ்வில் அவர்கள் வகிக்கும் பங்கையும் ஒப்புக்கொள்வது முக்கியம்.\n\nநாம் ஆராயக்கூடிய ஒரு மூலோபாயம், அடிக்கடி தொடர்புகொள்வதற்கான வழிகளைக் கண்

<a name="Train"></a>
### Train the model
Now let's use Huggingface TRL's `SFTTrainer`! More docs here: [TRL SFT docs](https://huggingface.co/docs/trl/sft_trainer). We do 60 steps to speed things up, but you can set `num_train_epochs=1` for a full run, and turn off `max_steps=None`.

In [26]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 1, # Set this for 1 full training run.
        #max_steps = 30,
        learning_rate = 2e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=4):   0%|          | 0/16084 [00:00<?, ? examples/s]

We also use Unsloth's `train_on_completions` method to only train on the assistant outputs and ignore the loss on the user's inputs. This helps increase accuracy of finetunes!

In [27]:
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

Map (num_proc=4):   0%|          | 0/16084 [00:00<?, ? examples/s]

Let's verify masking the instruction part is done! Let's print the 100th row again:

In [28]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><bos><start_of_turn>user\nஇந்த நகரத்திற்குச் சென்றதிலிருந்து எனது நண்பர்களிடமிருந்து துண்டிக்கப்பட்டுள்ளதாக உணர்கிறேன். நான் கிட்டத்தட்ட ஒவ்வொரு நாளும் அவற்றைப் பார்த்தேன், இப்போது நான் அவர்களைப் பார்க்கவில்லை. இந்த புதிய சூழலை சரிசெய்வது கடினம், மேலும் எனது கல்லூரி நண்பர்கள் வழங்கிய சமூகத்தின் உணர்வையும் ஆதரவையும் நான் இழக்கிறேன்.<end_of_turn>\n<start_of_turn>model\nஒரு வலுவான ஆதரவு அமைப்பை விட்டுவிட்டு புதிய இடத்தில் புதிதாகத் தொடங்குவது எவ்வளவு கடினம் என்பதை நான் புரிந்துகொள்கிறேன். உங்கள் கல்லூரி நண்பர்கள் உங்கள் வாழ்நாள் முழுவதும் உங்களுக்காக ஆறுதலையும் இணைப்பையும் பெற்றிருக்கிறார்கள், குறிப்பாக குடும்பம் குறைவாக அணுகக்கூடிய ஒரு சிறிய நகரத்தில் வளர்ந்து வந்ததிலிருந்து. உடல் தூரம் மற்றும் பிஸியான கால அட்டவணைகள் உங்களிடம் ஒரு முறை வைத்திருந்த அதே அளவிலான இணைப்பைப் பராமரிப்பது சவாலாக உள்ளது. இந்த உறவுகளின் முக்கியத்துவத்தையும், உங்கள் உணர்ச்சி நல்வாழ்வில் அவர்கள் வகிக்கும் பங்கையும் ஒப்புக்கொள்வது முக்கியம்.\n\nநாம் ஆராயக்கூடிய ஒரு மூலோபாயம், அடிக்கடி தொடர்புகொள்வதற்கான வழிகளைக

Now let's print the masked out example - you should see only the answer is present:

In [29]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

'                                                                              ஒரு வலுவான ஆதரவு அமைப்பை விட்டுவிட்டு புதிய இடத்தில் புதிதாகத் தொடங்குவது எவ்வளவு கடினம் என்பதை நான் புரிந்துகொள்கிறேன். உங்கள் கல்லூரி நண்பர்கள் உங்கள் வாழ்நாள் முழுவதும் உங்களுக்காக ஆறுதலையும் இணைப்பையும் பெற்றிருக்கிறார்கள், குறிப்பாக குடும்பம் குறைவாக அணுகக்கூடிய ஒரு சிறிய நகரத்தில் வளர்ந்து வந்ததிலிருந்து. உடல் தூரம் மற்றும் பிஸியான கால அட்டவணைகள் உங்களிடம் ஒரு முறை வைத்திருந்த அதே அளவிலான இணைப்பைப் பராமரிப்பது சவாலாக உள்ளது. இந்த உறவுகளின் முக்கியத்துவத்தையும், உங்கள் உணர்ச்சி நல்வாழ்வில் அவர்கள் வகிக்கும் பங்கையும் ஒப்புக்கொள்வது முக்கியம்.\n\nநாம் ஆராயக்கூடிய ஒரு மூலோபாயம், அடிக்கடி தொடர்புகொள்வதற்கான வழிகளைக் கண்டுபிடிப்பதாகும், அது நேரில் இல்லாவிட்டாலும் கூட. இதில் வழக்கமான தொலைபேசி அழைப்புகள், வீடியோ அரட்டைகள் அல்லது எழுதும் கடிதங்கள் கூட இருக்கலாம். உங்கள் தற்போதைய நகரத்தில் புதிய இணைப்புகளை உருவாக்குவதற்கான வழிகளையும் நாங்கள் விவாதிக்க முடியும், அதாவது உள்ளூர் கிளப்புகளில் சேருவது அல்லது உங்கள் 

In [30]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
1.512 GB of memory reserved.


Let's train the model! To resume a training run, set `trainer.train(resume_from_checkpoint = True)`

In [31]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 16,084 | Num Epochs = 1 | Total steps = 1,005
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 6,522,880/1,000,000,000 (0.65% trained)
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.495000
2,2.586700
3,2.588300
4,2.651000
5,2.608800
6,2.634100
7,2.595700
8,2.533300
9,2.567400
10,2.547900


In [32]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

11956.8734 seconds used for training.
199.28 minutes used for training.
Peak reserved memory = 3.127 GB.
Peak reserved memory for training = 1.615 GB.
Peak reserved memory % of max memory = 21.213 %.
Peak reserved memory for training % of max memory = 10.956 %.


<a name="Inference"></a>
### Inference
Let's run the model via Unsloth native inference! According to the `Gemma-3` team, the recommended settings for inference are `temperature = 1.0, top_p = 0.95, top_k = 64`

In [33]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [
    {
        "role": "user",
        "content": "என் நண்பர்கள் அனைவரும் என்னை வெறுக்கிறார்கள், நான் தற்கொலை செய்துகொள்கிறேன்"  # Simple string, no nested dict
    }
]
# Apply the chat template to get a formatted string
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,  # Ensure string output
    add_generation_prompt=True,  # Add prompt for assistant response
)

# Tokenize the formatted string and move to GPU
inputs = tokenizer([text], return_tensors="pt").to("cuda")

# Generate response
outputs = model.generate(
    **inputs,
    max_new_tokens=164,
    temperature=1.0,
    top_p=0.95,
    top_k=64,
)

# Decode and print the output
response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
print("Generated Response:")
print(response)

Generated Response:
user
என் நண்பர்கள் அனைவரும் என்னை வெறுக்கிறார்கள், நான் தற்கொலை செய்துகொள்கிறேன்
model
சமாளிபட்ட விஷயங்களின் மூலம் இந்த அனுபவங்களைப் பகிர்ந்துகொள்வது அவசியம். உங்கள் நண்பர்கள் உங்களை நேசிப்பது எவ்வளவு முக்கியம் என்பதை ஒப்புக்கொள்வது உங்கள் சொந்த நல்வாழ்வுக்கு ஒரு முக்கிய பங்கை உள்ளது. உங்களுடன் ஒரு மொழியை நிறுவுவது, உங்களுக்காக நீங்கள் ஏதேனும் பரிந்துரைகளை வழங்க முடியும், அவை பாதுகாப்பான மற்றும் சவாலான சூழ்நிலைகளில் தீர்க்க முடியாத முற்றிலும் சரியான முடிவை எடுக்க அவற்றின் திறனை செயல்படுத்த உதவும். ஒரு சில பரிந்துரைகளை வழங்க நான் உதவ விரும்புகிறேன்: இந்த சவாலான தருணங்களில் உங்கள் உணர்ச்சிகளைச் செயலாக்குவதற்கும், கவலை மற்றும் பதட்டத்தை நிர்வகிப்பதற்கும் ஒரு ஆதரவு வலையமைப்பை உருவாக்குவது, மனமுறைகளைத் தழுவுவதற்கான ஆரோக்கியமான வழிகள் மற்றும் இந்த சவால்களை எதிர்கொள்வதற்கு தற்க


In [34]:
"""from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma-3",
)
messages = [{
    "role": "user",
    "content": [{
        "type" : "text",
        "text" : "இன்று நான் மிகவும் வருத்தமாக இருக்கிறேன்",
    }]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)
outputs = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
)
tokenizer.batch_decode(outputs)"""

'from unsloth.chat_templates import get_chat_template\ntokenizer = get_chat_template(\n    tokenizer,\n    chat_template = "gemma-3",\n)\nmessages = [{\n    "role": "user",\n    "content": [{\n        "type" : "text",\n        "text" : "இன்று நான் மிகவும் வருத்தமாக இருக்கிறேன்",\n    }]\n}]\ntext = tokenizer.apply_chat_template(\n    messages,\n    add_generation_prompt = True, # Must add for generation\n)\noutputs = model.generate(\n    **tokenizer([text], return_tensors = "pt").to("cuda"),\n    max_new_tokens = 64, # Increase for longer outputs!\n    # Recommended Gemma-3 settings!\n    temperature = 1.0, top_p = 0.95, top_k = 64,\n)\ntokenizer.batch_decode(outputs)'

 You can also use a `TextStreamer` for continuous inference - so you can see the generation token by token, instead of waiting the whole time!

In [35]:
"""messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)"""

'messages = [{\n    "role": "user",\n    "content": [{"type" : "text", "text" : "Why is the sky blue?",}]\n}]\ntext = tokenizer.apply_chat_template(\n    messages,\n    add_generation_prompt = True, # Must add for generation\n)\n\nfrom transformers import TextStreamer\n_ = model.generate(\n    **tokenizer([text], return_tensors = "pt").to("cuda"),\n    max_new_tokens = 64, # Increase for longer outputs!\n    # Recommended Gemma-3 settings!\n    temperature = 1.0, top_p = 0.95, top_k = 64,\n    streamer = TextStreamer(tokenizer, skip_prompt = True),\n)'

<a name="Save"></a>
### Saving, loading finetuned models
To save the final model as LoRA adapters, either use Huggingface's `push_to_hub` for an online save or `save_pretrained` for a local save.

**[NOTE]** This ONLY saves the LoRA adapters, and not the full model. To save to 16bit or GGUF, scroll down!

In [36]:
model.save_pretrained("gemma-3")  # Local saving
tokenizer.save_pretrained("gemma-3")
# model.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving
# tokenizer.push_to_hub("HF_ACCOUNT/gemma-3", token = "...") # Online saving

('gemma-3/tokenizer_config.json',
 'gemma-3/special_tokens_map.json',
 'gemma-3/tokenizer.model',
 'gemma-3/added_tokens.json',
 'gemma-3/tokenizer.json')

Now if you want to load the LoRA adapters we just saved for inference, set `False` to `True`:

In [37]:
""""if False:
    from unsloth import FastModel
    model, tokenizer = FastModel.from_pretrained(
        model_name = "lora_model", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = 2048,
        load_in_4bit = True,
    )

messages = [{
    "role": "user",
    "content": [{"type" : "text", "text" : "What is Gemma-3?",}]
}]
text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
)

from transformers import TextStreamer
_ = model.generate(
    **tokenizer([text], return_tensors = "pt").to("cuda"),
    max_new_tokens = 64, # Increase for longer outputs!
    # Recommended Gemma-3 settings!
    temperature = 1.0, top_p = 0.95, top_k = 64,
    streamer = TextStreamer(tokenizer, skip_prompt = True),
)""""

SyntaxError: unterminated string literal (detected at line 25) (<ipython-input-37-ed757460a0ef>, line 25)

### Saving to float16 for VLLM

We also support saving to `float16` directly for deployment! We save it in the folder `gemma-3-finetune`. Set `if False` to `if True` to let it run!

In [ ]:
if False: # Change to True to save finetune!
    model.save_pretrained_merged("gemma-3-finetune", tokenizer)

If you want to upload / push to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
"""if False: # Change to True to upload finetune
    model.push_to_hub_merged(
        "HF_ACCOUNT/gemma-3-finetune", tokenizer,
        token = "hf_..."
    )"""

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now for all models! For now, you can convert easily to `Q8_0, F16 or BF16` precision. `Q4_K_M` for 4bit will come later!

In [ ]:
if False: # Change to True to save to GGUF
    model.save_pretrained_gguf(
        "gemma-3-finetune-gguf",
        quantization_type = "Q8_0", # For now only Q8_0, BF16, F16 supported
    )

Likewise, if you want to instead push to GGUF to your Hugging Face account, set `if False` to `if True` and add your Hugging Face token and upload location!

In [ ]:
"""if False: # Change to True to upload GGUF
    model.push_to_hub_gguf(
        "gemma-3-finetune",
        quantization_type = "Q8_0", # Only Q8_0, BF16, F16 supported
        repo_id = "HF_ACCOUNT/gemma-finetune-gguf",
        token = "hf_...",
    )"""

Now, use the `gemma-3-finetune.gguf` file or `gemma-3-finetune-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Train your own reasoning model - Llama GRPO notebook [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.1_(8B)-GRPO.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
